In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/pctl_idual
!ls

/content/drive/MyDrive/pctl_idual
dp_solvers.py  idual_solvers.py  lp_solvers.py	  __pycache__
gridworld.py   __init__.py	 pctl_solvers.py


In [ ]:
import os
os.makedirs('/root/mosek', exist_ok=True)

!cp "/content/drive/MyDrive/mosek/mosek.lic" "/root/mosek/mosek.lic"

print("Copied license to /root/mosek/mosek.lic")

!pip install mosek

Copied license to /root/mosek/mosek.lic
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 9.5 MB/s eta 0:00:00


In [ ]:
import cvxpy as cp

x = cp.Variable()
y = cp.Variable()

prob = cp.Problem(
    cp.Minimize(x + y),
    [
        x + 2*y >= 1,
        x >= 0,
        y >= 0,
    ]
)

prob.solve(solver=cp.MOSEK)

print("status:", prob.status)
print("x,y =", x.value, y.value)
print("objective:", prob.value)


status: optimal
x,y = 0.0 0.5
objective: 0.5


In [ ]:
print(cp.installed_solvers())


['CLARABEL', 'CVXOPT', 'GLPK', 'GLPK_MI', 'HIGHS', 'MOSEK', 'OSQP', 'SCIPY', 'SCS']


# Global LP

In [ ]:
import sys, os
import time
#parent of pctl_idual
sys.path.append("/content/drive/MyDrive")

# print("Top-level MyDrive:", os.listdir("/content/drive/MyDrive"))
# print("Inside pctl_idual:", os.listdir("/content/drive/MyDrive/pctl_idual"))

import importlib
# Reload in case you edited the files
import pctl_idual.gridworld as gw
import pctl_idual.dp_solvers as dp
import pctl_idual.lp_solvers as lp
import pctl_idual.pctl_solvers as ps

importlib.reload(gw)
importlib.reload(dp)
importlib.reload(lp)
importlib.reload(ps)

from pctl_idual.gridworld import (
    make_grid_world,
    print_cost_grid,
    make_4x4_world,
    make_4x4_pctl_world,
    make_20x20_world,
    make_NxN_world
)

from pctl_idual.dp_solvers import (
    value_iteration_shortest_path,
    greedy_policy_from_V,
    simulate_policy,
)
from pctl_idual.lp_solvers import (
    solve_shortest_path_lp,
    recover_policy_from_x,
    print_policy_grid
)

from pctl_idual.pctl_solvers import (
    RegionFlagSpec,
    PCTLRegionConstraint,
    UntilSpec,
    UntilConstraint,
    AugmentedMDP,
    solve_lp_with_pctl_aug,
    recover_policy_from_x_aug,
    print_policy_grid_z0,
    print_policy_for_flags,
    simulate_policy_aug
)

def collapse_augmented_policy_to_base(mdp_aug, policy_aug):
    """
    For each base state s, combine action probabilities
    across *all* flag valuations into one base policy.
    (simple average over reachable augmented states)
    """
    base_policy = {s: {} for s in mdp_aug.base.states}

    for st_aug, act_probs in policy_aug.items():
        s = st_aug[0]  # physical state
        for a, p in act_probs.items():
            base_policy[s][a] = base_policy[s].get(a, 0.0) + p

    # normalize each state's probs
    for s, ap in base_policy.items():
        total = sum(ap.values())
        if total > 0:
            for a in ap:
                ap[a] /= total

    return base_policy


# ------- choose which world to use -------

WORLD = "experiment"      # options: "4x4", "20x20", "custom"
USE_PCTL = (WORLD == "experiment")

if WORLD == "4x4_pctl":
    # Base MDP
    mdp = make_4x4_pctl_world()

    # Define A and B regions for A U B

    G2 = {(r, c) for r in range(2, 3) for c in range(0, 2)}  # row 2, cols 0–1
    G3 = {(r, c) for r in range(1, 2) for c in range(0, 2)}  # row 1, cols 0–1
    # G2 = {(0, c) for c in range(4)} | {(r, 0) for r in range(4)}
    # G4 = {(r, c) for r in range(2, 4) for c in range(1, 4)}
    # G3 = {(r, c) for r in range(1, 2) for c in range(1, 2)}  #

    flags = [
        RegionFlagSpec("G2", G2),
        RegionFlagSpec("G4", G4),
        RegionFlagSpec("G3", G3),
    ]
    until_specs = [
        UntilSpec("G2U_G3", A_region=G2, B_region=G3),
    ]

    mdp_aug = AugmentedMDP(mdp, flags=flags, until_specs=until_specs)

    p_goal_min = 1  # force eventually reaching the main goal with prob 1

    region_constraints = [
        #P(ever visit G2) ≤ 0   (hard avoidance)
        # PCTLRegionConstraint("visit_region_min", "G3", 1)
        # PCTLRegionConstraint("visit_region_max", "G4", 0),
        # PCTLRegionConstraint("visit_region_max", "G4", 1),
    ]

    until_constraints = [
        # P >= 0.7 [ G2 U G3 ]
        # UntilConstraint("until_min", "G2U_G3", 0.7),
        # UntilConstraint("until_min", "G2U_G3", 1)
    ]

    J_pctl, p_goal, x_opt_aug, region_probs, until_probs, t_pctl = solve_lp_with_pctl_aug(
        mdp_aug,
        p_goal_min=p_goal_min,
        region_constraints=region_constraints,
        until_constraints=until_constraints,
    )

    if x_opt_aug is None:
        print("\n[Global PCTL+Until LP] No feasible policy.")
    else:
        print("\n=== Global LP with PCTL + Until ===")
        print("Optimal expected cost:", float(J_pctl))
        print("P(reach GOAL):", float(p_goal))
        for name, val in region_probs.items():
            print(f"P(ever visit {name}):", float(val))
        for name, val in until_probs.items():
            print(f"P({name}):", float(val))
        print(f"Time taken to solve dual LP:", round(t_pctl, 3), "s")

        policy_aug = recover_policy_from_x_aug(mdp_aug, x_opt_aug)
        base_traj_pctl, aug_traj_pctl = simulate_policy_aug(mdp_aug, policy_aug)
        print("\nTrajectory under PCTL-constrained policy (base states):", base_traj_pctl)

        # total_bits = len(flags) + 2 * len(until_specs)

        # print("\nPolicy at flags = all zero (start semantics)")
        # print_policy_for_flags(
        #     mdp_aug,
        #     policy_aug,
        #     flag_tuple=(0,) * total_bits
        # )

        # Example: “inside G2, ongoing A U B”
        # Here flags layout is: [G2][G2U_G3_success][G2U_G3_fail]
        # print("\nPolicy when already in G2, not yet hit B:")
        # print_policy_for_flags(mdp_aug, policy_aug, flag_tuple=(1, 0, 0))

        # print("\nPolicy after success of G2 U G3:")
        # print_policy_for_flags(mdp_aug, policy_aug, flag_tuple=(1, 1, 0))

        base_policy = collapse_augmented_policy_to_base(mdp_aug, policy_aug)

        print("\nFinal policy (collapsed to base MDP):")
        print_policy_grid(mdp, base_policy)

elif WORLD == "20x20_pctl":

    mdp = make_20x20_world()
    goal = {(0, 19)}
    N = 20

    G_corridor = set()

    # bottom horizontal segment: (19,0) to (19,5)
    for c in range(0, 6):
        G_corridor.add((19, c))

    # vertical segment: (18,5) up to (11,5)
    for r in range(18, 10, -1):   # 18,17,...,11
        G_corridor.add((r, 5))

    G2 = G_corridor
    G3 = {(11, 5)}


    # G2 = {(r, c) for r in range(8, 12) for c in range(16, 20)}
    # G3 = {(r, c) for r in range(0, 8)  for c in range(0, 8)}

    flags = [
        RegionFlagSpec("G2", G2),
        RegionFlagSpec("G3", G3),

    ]
    until_specs = [
        UntilSpec("G2U_G3", A_region=G2, B_region=G3),

    ]

    mdp_aug = AugmentedMDP(mdp, flags=flags, until_specs=until_specs)
    print(f"Total augmented states (|S_aug|): {len(mdp_aug.states_aug)}")


    p_goal_min = 1 # force eventually reaching the main goal with prob 1

    region_constraints = [
        # P(ever visit G3) ≤ 0   (hard avoidance)
        # PCTLRegionConstraint("visit_region_min", "G2", 1),
        # PCTLRegionConstraint("visit_region_min", "G3", 0.5),
    ]

    until_constraints = [
        # P >= 0.7 [ G2 U G3 ]
        UntilConstraint("until_min", "G2U_G3", 1),
    ]

    J_pctl, p_goal, x_opt_aug, region_probs, until_probs, t_pctl = solve_lp_with_pctl_aug(
        mdp_aug,
        p_goal_min=p_goal_min,
        region_constraints=region_constraints,
        until_constraints=until_constraints,
    )

    if x_opt_aug is None:
        print("\n[Global PCTL+Until LP] No feasible policy.")
    else:
        print("\n=== Global LP with PCTL + Until ===")
        print("Optimal expected cost:", float(J_pctl))
        print("P(reach GOAL):", float(p_goal))
        for name, val in region_probs.items():
            print(f"P(ever visit {name}):", float(val))
        for name, val in until_probs.items():
            print(f"P({name}):", float(val))
        print(f"Time taken to solve dual LP:", round(t_pctl, 3), "s")

        policy_aug = recover_policy_from_x_aug(mdp_aug, x_opt_aug)
        base_traj_pctl, aug_traj_pctl = simulate_policy_aug(mdp_aug, policy_aug)
        print("\nTrajectory under PCTL-constrained policy (base states):", base_traj_pctl)

        base_policy = collapse_augmented_policy_to_base(mdp_aug, policy_aug)

        print("\nFinal policy (collapsed to base MDP):")
        print_policy_grid(mdp, base_policy, G2=G2, G3=G3)



elif WORLD == "4x4":
    mdp = make_4x4_world()

elif WORLD == "20x20":
    mdp = make_20x20_world()

elif WORLD == "custom":
    # Example custom config:
    N = 50
    start = (49, 0)
    goal = {(0, 49)}

    rect_costs = [
        (16, 16, 19, 19, 0.1),  # cheap block
        (5, 5, 14, 14, 10.0),   # expensive block
    ]

    mdp = make_grid_world(
        N=N,
        start=start,
        goal=goal,
        default_cost=1.0,
        rect_costs=rect_costs,
        slip_prob=0.0
    )


elif WORLD == "experiment":
    Ns = [5, 20, 30, 40, 50, 70, 100, 150, 200, 250]
    # Ns = [5, 7, 10, 15, 20, 25]
    # Example custom config:
    for N in Ns:
      print("\n==============================")
      print(f"Experiment for N = {N}")
      print("==============================")
      mdp = make_NxN_world(N)

      goal = {(0, N-1)}
      G2 = {
        (r,c)
        for r in range(N)
        for c in range(N)
        if (r, c) not in goal
      }
      G3 = goal
      flags = [
        RegionFlagSpec("G2", G2),
        RegionFlagSpec("G3", G3),

      ]
      until_specs = [
        UntilSpec("G2U_G3", A_region=G2, B_region=G3),

      ]

      mdp_aug = AugmentedMDP(mdp, flags=flags, until_specs=until_specs)
      # basic size info
      S_aug = len(mdp_aug.states_aug)
      non_abs = sum(
            1 for st in mdp_aug.states_aug
            if not mdp_aug.is_absorbing_aug(st)
      )
      print(f"|S_base| = {N*N}")
      print(f"|S_aug|  = {S_aug}")
      print(f"non-absorbing aug states: {non_abs}")

      p_goal_min = 1

      region_constraints = [

      ]

      until_constraints = [
        # P >= 1 [ G2 U G3 ]
        UntilConstraint("until_min", "G2U_G3", 1),
      ]
      t0 = time.perf_counter()
      J_pctl, p_goal, x_opt_aug, region_probs, until_probs, t_pctl = solve_lp_with_pctl_aug(
      mdp_aug,
      p_goal_min=p_goal_min,
      region_constraints=region_constraints,
      until_constraints=until_constraints,
      )
      t1 = time.perf_counter()
      time_global = t1 - t0
       # 6) Count the number of x-variables
      # If x_opt_aug is a dict: {(st,a): value}
      if isinstance(x_opt_aug, dict):
          num_x = len(x_opt_aug)
      else:
          # if it's a numpy array or cvxpy vector:
          num_x = x_opt_aug.size

      print(f"global LP x-vars (edges):   {num_x}")
      print(f"optimization time:      {t_pctl:.4f} s")
      print(f"full pipeline time:  {time_global:.4f} s")
      print(f"P(reach GOAL):             {p_goal}")
      print(f"P(G2U_G3):                 {until_probs.get('G2U_G3')}")
      print(f"Objective value:                 {J_pctl:.4f}")


else:
    raise ValueError(f"Unknown WORLD='{WORLD}'")

if not USE_PCTL:
  # DP ground truth
  V = value_iteration_shortest_path(mdp)
  print("DP (unconstrained) optimal cost from START:", V[mdp.start])

  pi = greedy_policy_from_V(mdp, V)
  traj = simulate_policy(mdp, pi)
  print("Trajectory under DP (unconstrained):", traj)
  # LP solution
  J_lp, x_opt, t_lp = solve_shortest_path_lp(mdp)
  print("LP cost:", J_lp, "   solve time:", t_lp)

  pi_lp = recover_policy_from_x(mdp, x_opt)
  print("\nPolicy from LP:")
  print_policy_grid(mdp, pi_lp)



Experiment for N = 5
|S_base| = 25
|S_aug|  = 400
non-absorbing aug states: 384
=== Global LP size (PCTL+until) ===
  non-absorbing aug states : 384
  edges / x-vars           : 1536
  constraints              : 3
global LP x-vars (edges):   1536
optimization time:      0.0260 s
full pipeline time:  0.0424 s
P(reach GOAL):             1.0
P(G2U_G3):                 1.0
Objective value:                 1.7000

Experiment for N = 20
|S_base| = 400
|S_aug|  = 6400
non-absorbing aug states: 6384
=== Global LP size (PCTL+until) ===
  non-absorbing aug states : 6384
  edges / x-vars           : 25536
  constraints              : 3
global LP x-vars (edges):   25536
optimization time:      3.8098 s
full pipeline time:  4.1035 s
P(reach GOAL):             1.0000000002324219
P(G2U_G3):                 1.0000000000881024
Objective value:                 31.7000

Experiment for N = 30
|S_base| = 900
|S_aug|  = 14400
non-absorbing aug states: 14384
=== Global LP size (PCTL+until) ===
  non-absorbi

# I-dual

In [ ]:
import sys, os
import time
#parent of pctl_idual
sys.path.append("/content/drive/MyDrive")

# print("Top-level MyDrive:", os.listdir("/content/drive/MyDrive"))
# print("Inside pctl_idual:", os.listdir("/content/drive/MyDrive/pctl_idual"))

import importlib
# Reload in case you edited the files
import pctl_idual.gridworld as gw
import pctl_idual.dp_solvers as dp
import pctl_idual.lp_solvers as lp
import pctl_idual.pctl_solvers as ps

importlib.reload(gw)
importlib.reload(dp)
importlib.reload(lp)
importlib.reload(ps)

from pctl_idual.gridworld import (
    make_grid_world,
    print_cost_grid,
    make_4x4_world,
    make_4x4_pctl_world,
    make_20x20_world,
    make_NxN_world
)

from pctl_idual.dp_solvers import (
    value_iteration_shortest_path,
    greedy_policy_from_V,
    simulate_policy,
)
from pctl_idual.lp_solvers import (
    solve_shortest_path_lp,
    recover_policy_from_x,
    print_policy_grid
)

from pctl_idual.pctl_solvers import (
    RegionFlagSpec,
    PCTLRegionConstraint,
    UntilSpec,
    UntilConstraint,
    AugmentedMDP,
    solve_lp_with_pctl_aug,
    recover_policy_from_x_aug,
    print_policy_grid_z0,
    print_policy_for_flags,
    simulate_policy_aug
)
import importlib

# ---- i-dual solvers ----
import pctl_idual.idual_solvers as ids
importlib.reload(ids)

from pctl_idual.idual_solvers import (
    compute_in_flow_aug,
    solve_lp3_aug,
    i_dual_aug,
    i_dual_aug_trevizan,
    compute_max_prob_visit_region_before_goal,
    compute_min_prob_visit_region_before_goal,
    compute_max_prob_until_before_goal,
    compute_min_prob_until_before_goal,
)

WORLD = "experiment"

if WORLD == "4x4_pctl_idual":
    # 1) Base MDP
    mdp = make_4x4_pctl_world()

    # 2) Define regions
    G2 = {(r, c) for r in range(2, 3) for c in range(0, 2)}  # row 2, cols 0–1
    G3 = {(r, c) for r in range(1, 2) for c in range(0, 2)}  # row 1, cols 0–1
    # G2 = {(0, c) for c in range(4)} | {(r, 0) for r in range(4)}
    # G4 = {(r, c) for r in range(2, 4) for c in range(1, 4)}
    # G3 = {(r, c) for r in range(1, 2) for c in range(1, 2)}  #





    # 3) Flags and until specs
    flags = [
        RegionFlagSpec("G2", G2),
        RegionFlagSpec("G4", G4),
        RegionFlagSpec("G3", G3),

    ]

    until_specs = [
        UntilSpec("G2U_G3", A_region=G2, B_region=G3),
    ]

    mdp_aug = AugmentedMDP(mdp, flags=flags, until_specs=until_specs)

    region_constraints = [
        # P(ever visit G4) ≤ 0   (hard avoidance of G4)
        # PCTLRegionConstraint("visit_region_max", "G4", 0.5),
        # PCTLRegionConstraint("visit_region_min", "G3", 0.5),

    ]

    until_constraints = [
        # P >= 0.7 [ G2 U G3 ]
        UntilConstraint("until_min", "G2U_G3", 1),
        # UntilConstraint("until_min", "G2U_G3", 1.0)
    ]
    constraints_region = region_constraints
    constraints_full = region_constraints + until_constraints  # for global LP

    p_goal_min = 1
    # 1) Unconstrained DP on the base MDP
    V_base = value_iteration_shortest_path(mdp)  # returns optimal cost-to-go from each base state

    H_cost = {st_aug: V_base[st_aug[0]] for st_aug in mdp_aug.states_aug}

    H_region_upper = {
    # "G4": compute_min_prob_visit_region_before_goal(mdp_aug, "G4")
    }

    H_region_lower = {
    "G3": compute_max_prob_visit_region_before_goal(mdp_aug, "G3")
    }

    H_until_lower = {
    # used for until_min (P >= bound)
    "G2U_G3": compute_max_prob_until_before_goal(mdp_aug, "G2U_G3")
    }
    H_until_upper = {
    # used for until_min (P >= bound)
    "G2U_G3": compute_min_prob_until_before_goal(mdp_aug, "G2U_G3")
    }

    H_max = compute_max_prob_until_before_goal(mdp_aug, "G2U_G3")
    p_star = H_max[mdp_aug.start_aug]
    print(p_star)


    # 2) Build heuristic on augmented states
    H = {}
    for st_aug in mdp_aug.states_aug:
        s = st_aug[0]  # physical state
        H[st_aug] = V_base.get(s, 0.0)

    (
        x_final,
        goal_prob_final,
        region_flag_final,
        until_final,
        total_time,
        final_lp_time,
        n_iters,
        envelope_size,
        obj_final
    ) = i_dual_aug_trevizan(
        mdp_aug,
        p_goal_min=p_goal_min,
        extra_constraints=constraints_full,
        H=H_cost,
        H_region_upper=H_region_upper,
        H_region_lower = H_region_lower,
        H_until_upper=H_until_upper,
        H_until_lower=H_until_lower,
    )

    if x_final is None:
      print("\n[i-dual] No feasible policy under these constraints.")
    else:
      from collections import defaultdict
      flow_per_base = defaultdict(float)
      for (st, a), val in x_final.items():
        if val is None:
            continue
        (r, c) = st[0]
        flow_per_base[(r, c)] += float(val)



    print("\n=== timing summary ===")
    print("total i-dual time:", total_time)
    print("final LP time:", final_lp_time)
    print("iterations:", n_iters)
    print("final envelope size:", envelope_size)
    for name, val in region_flag_final.items():
      print(f"[i-dual] P(ever visit {name}):", float(val))
    for name, val in until_final.items():
      print(f"[i-dual] P({name}):", float(val))

    print("P(reach GOAL):", float(goal_prob_final))
    print("i-dual objective:", obj_final)


    policy_aug = recover_policy_from_x_aug(mdp_aug, x_final)
    base_traj_pctl, aug_traj_pctl = simulate_policy_aug(mdp_aug, policy_aug)
    print("\nTrajectory under PCTL-constrained policy (base states):", base_traj_pctl)

    base_policy = collapse_augmented_policy_to_base(mdp_aug, policy_aug)

    print("\nFinal policy (collapsed to base MDP):")
    print_policy_grid(mdp, base_policy, G2=G2, G3=G3)

elif WORLD == "20x20_pctl_idual":
    # 1) Base MDP
    mdp = make_20x20_world()
    # G2 = {(r, c) for r in range(8, 12) for c in range(16, 20)}
    # G3 = {(r, c) for r in range(0, 8)  for c in range(0, 8)}

    G_corridor = set()

    # bottom horizontal segment: (19,0) to (19,5)
    for c in range(0, 6):
        G_corridor.add((19, c))

    # vertical segment: (18,5) up to (11,5)
    for r in range(18, 10, -1):   # 18,17,...,11
        G_corridor.add((r, 5))

    G2 = G_corridor
    G3 = {(11, 5)}

    flags = [
        RegionFlagSpec("G2", G2),
        RegionFlagSpec("G3", G3),
    ]
    until_specs = [
        UntilSpec("G2U_G3", A_region=G2, B_region=G3),

    ]

    mdp_aug = AugmentedMDP(mdp, flags=flags, until_specs=until_specs)
    print(f"Total augmented states (|S_aug|): {len(mdp_aug.states_aug)}")

    p_goal_min = 1 # force eventually reaching the main goal with prob 1

    region_constraints = [
        # P(ever visit G3) ≤ 0   (hard avoidance)
        # PCTLRegionConstraint("visit_region_min", "G2", 1),
    ]

    until_constraints = [
        # P >= 0.7 [ G2 U G3 ]
        UntilConstraint("until_min", "G2U_G3", 1),
    ]

    constraints_full = region_constraints + until_constraints  # for global LP

    p_goal_min = 1
    # 1) Unconstrained DP on the base MDP
    V_base = value_iteration_shortest_path(mdp)  # returns optimal cost-to-go from each base state

    H_cost = {st_aug: V_base[st_aug[0]] for st_aug in mdp_aug.states_aug}

    H_region_upper = {
    # "G2": compute_min_prob_visit_region_before_goal(mdp_aug, "G2")

    }

    H_region_lower = {
    # "G2": compute_max_prob_visit_region_before_goal(mdp_aug, "G2"),
    }

    H_until_lower = {
    # used for until_min (P >= bound)
    "G2U_G3": compute_max_prob_until_before_goal(mdp_aug, "G2U_G3")
    }

    H_until_upper = {
        # "G2U_G3": compute_min_prob_until_before_goal(mdp_aug, "G2U_G3")
    }


    (
        x_final,
        goal_prob_final,
        region_flag_final,
        until_final,
        total_time,
        final_lp_time,
        n_iters,
        envelope_size,
        obj_final
    ) = i_dual_aug_trevizan(
        mdp_aug,
        p_goal_min=p_goal_min,
        extra_constraints=constraints_full,
        H=H_cost,
        H_region_upper=H_region_upper,
        H_region_lower = H_region_lower,
        H_until_upper=H_until_upper,
        H_until_lower=H_until_lower,
    )

    if x_final is None:
      print("\n[i-dual] No feasible policy under these constraints.")
    else:
      from collections import defaultdict
      flow_per_base = defaultdict(float)
      for (st, a), val in x_final.items():
        if val is None:
            continue
        (r, c) = st[0]
        flow_per_base[(r, c)] += float(val)



    print("\n=== timing summary ===")
    print("total i-dual time:", total_time)
    print("final LP time:", final_lp_time)
    print("iterations:", n_iters)
    print("final envelope size:", envelope_size)
    print(f"Total augmented states: {len(mdp_aug.states_aug)}")
    print(f"Fraction explored:      {envelope_size / len(mdp_aug.states_aug):.4f}")

    for name, val in region_flag_final.items():
      print(f"[i-dual] P(ever visit {name}):", float(val))
    for name, val in until_final.items():
      print(f"[i-dual] P({name}):", float(val))

    print("P(reach GOAL):", float(goal_prob_final))
    print("i-dual objective:", obj_final)


    policy_aug = recover_policy_from_x_aug(mdp_aug, x_final)
    base_traj_pctl, aug_traj_pctl = simulate_policy_aug(mdp_aug, policy_aug)
    print("\nTrajectory under PCTL-constrained policy (base states):", base_traj_pctl)

    base_policy = collapse_augmented_policy_to_base(mdp_aug, policy_aug)

    print("\nFinal policy (collapsed to base MDP):")

    print_policy_grid(mdp, base_policy, G2=G2, G3=G3)

elif WORLD == "experiment":
    Ns = [5, 20, 30, 40, 50, 70, 100, 150, 200, 250]
    # Ns = [5, 7, 10, 15, 20, 25]
    # Example custom config:
    for N in Ns:
      t0 = time.perf_counter()
      print("\n==============================")
      print(f"Experiment for N = {N}")
      print("==============================")
      mdp = make_NxN_world(N)

      goal = {(0, N-1)}
      G2 = {
        (r,c)
        for r in range(N)
        for c in range(N)
        if (r, c) not in goal
      }
      G3 = goal
      flags = [
        RegionFlagSpec("G2", G2),
        RegionFlagSpec("G3", G3),

      ]
      until_specs = [
        UntilSpec("G2U_G3", A_region=G2, B_region=G3),

      ]

      mdp_aug = AugmentedMDP(mdp, flags=flags, until_specs=until_specs)
      # basic size info
      S_aug = len(mdp_aug.states_aug)
      non_abs = sum(
            1 for st in mdp_aug.states_aug
            if not mdp_aug.is_absorbing_aug(st)
      )
      print(f"|S_base| = {N*N}")
      print(f"|S_aug|  = {S_aug}")
      print(f"non-absorbing aug states: {non_abs}")

      p_goal_min = 1

      region_constraints = [

      ]

      until_constraints = [
        # P >= 1 [ G2 U G3 ]
        UntilConstraint("until_min", "G2U_G3", 1),
      ]


      constraints_full = region_constraints + until_constraints  # for global LP

      # 1) Unconstrained DP on the base MDP
      V_base = value_iteration_shortest_path(mdp)  # returns optimal cost-to-go from each base state

      H_cost = {st_aug: V_base[st_aug[0]] for st_aug in mdp_aug.states_aug}

      H_region_upper = {
      # "G2": compute_min_prob_visit_region_before_goal(mdp_aug, "G2")

      }

      H_region_lower = {
      # "G2": compute_max_prob_visit_region_before_goal(mdp_aug, "G2"),
      }

      H_until_lower = {
      # used for until_min (P >= bound)
      "G2U_G3": compute_max_prob_until_before_goal(mdp_aug, "G2U_G3")
      }

      H_until_upper = {
        # "G2U_G3": compute_min_prob_until_before_goal(mdp_aug, "G2U_G3")
      }

      (
        x_final,
        goal_prob_final,
        region_flag_final,
        until_final,
        total_time,
        final_lp_time,
        n_iters,
        envelope_size,
        obj_final
      ) = i_dual_aug_trevizan(
        mdp_aug,
        p_goal_min=p_goal_min,
        extra_constraints=constraints_full,
        H=H_cost,
        H_region_upper=H_region_upper,
        H_region_lower = H_region_lower,
        H_until_upper=H_until_upper,
        H_until_lower=H_until_lower,
    )
      t1 = time.perf_counter()
      time_global = t1 - t0
       # 6) Count the number of x-variables
      # If x_opt_aug is a dict: {(st,a): value}
      if isinstance(x_final, dict):
          num_x = len(x_final)
      else:
          # if it's a numpy array or cvxpy vector:
          num_x = x_final.size

      print(f"global LP x-vars (edges):   {num_x}")
      print(f"optimization time:      {total_time:.4f} s")
      print(f"full pipeline time:  {time_global:.4f} s")
      print(f"P(reach GOAL):             {goal_prob_final}")
      print(f"P(G2U_G3):                 {until_final.get('G2U_G3')}")
      print(f"Objective value:                 {obj_final:.4f}")
      # How many augmented & base states are actually used by the final policy?
      explored_aug_states = {
          st for (st, a) in x_final.keys()
          if float(x_final[(st, a)]) > 1e-9
      }
      num_explored_aug = len(explored_aug_states)

      explored_base_states = {st_aug[0] for st_aug in explored_aug_states}
      num_explored_base = len(explored_base_states)

      frac_aug_explored = num_explored_aug / len(mdp_aug.states_aug)
      frac_base_explored = num_explored_base / len(mdp.states)

      print(f"explored aug states (flow):  {num_explored_aug} / {len(mdp_aug.states_aug)} "
            f"({frac_aug_explored:.4%})")
      print(f"explored base states (flow): {num_explored_base} / {len(mdp.states)} "
            f"({frac_base_explored:.4%})")
      print(f"final envelope size (S_hat): {envelope_size} "
            f"({envelope_size / len(mdp_aug.states_aug):.4%} of S_aug)")



Experiment for N = 5
|S_base| = 25
|S_aug|  = 400
non-absorbing aug states: 384
global LP x-vars (edges):   68
optimization time:      0.3769 s
full pipeline time:  0.4040 s
P(reach GOAL):             1.0000000143904315
P(G2U_G3):                 1.0000000143904315
Objective value:                 1.7000
explored aug states (flow):  16 / 400 (4.0000%)
explored base states (flow): 16 / 25 (64.0000%)
final envelope size (S_hat): 25 (6.2500% of S_aug)

Experiment for N = 20
|S_base| = 400
|S_aug|  = 6400
non-absorbing aug states: 6384
global LP x-vars (edges):   188
optimization time:      4.2621 s
full pipeline time:  5.5454 s
P(reach GOAL):             0.9999999998994309
P(G2U_G3):                 0.9999999998994309
Objective value:                 31.7000
explored aug states (flow):  46 / 6400 (0.7188%)
explored base states (flow): 46 / 400 (11.5000%)
final envelope size (S_hat): 84 (1.3125% of S_aug)

Experiment for N = 30
|S_base| = 900
|S_aug|  = 14400
non-absorbing aug states: 143